# Experiment on the encoding and imputing technics for the Nam network, with and without Z ! 

In [38]:
run init.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done.


**`NAN` Second set of experiments deciding wheether `using indicator variable is useful`**

In [39]:
"""  Parameters of the experiment """
approach = 'xgboost'

range_dataset_name = ['circles', 'moons', 'blobs']
range_ratio_of_missing_values = [0., 0.1, 0.2, 0.3]
range_ratio_missing_per_class = [[0,0.25], [0.1, 0.3]]
range_imbalance_ratio = [0.5, 0.25, 0.1]
range_missingness_pattern = [1, 3, 4, 5]

range_missing_data_handling = ['without', 'encoding'] #TODO THIS IS WHERE WE SHOULD DO IMUTING BASED ON THE  OTHER EXPERIMENT
imputation_method = 'without'
range_use_missing_indicator_variables = [True, False]

In [40]:
if False:
    df = create_df(folder_names=['experiments_24_june_duke', 'experiments_24_june', 'experiments_solidays', 'experiement_28_june', 'experiments_28_june_duke'])
else:
    df = pd.read_csv("/home01/sam/df_all_experiments.csv")
    

In [41]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):
                        


                        if check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          approach=approach,
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                          missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class=ratio_missing_per_class):


                            print("Experiment already done.")
                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1):
                            print("Not doing this.")
                            continue
                            
                            
                            
                        print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                        missing_data_handling,
                                                                                                                                                          imputation_method, 
                                                                                                                                                          imbalance_ratio,
                                                                                                                                                          missingness_pattern,
                                                                                                                                                          ratio_of_missing_values, 
                                                                                                                                                              ratio_missing_per_class))



Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 4 ratio_missing:None ratio_per_class: [0, 0.25]
circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 4 ratio_missing:None ratio_per_class: [0.1, 0.3]
circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 5 ratio_missing:None ratio_per_class: [0, 0.25]
circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 5 ratio_missing:None ratio_per_class: [0.1, 0.3]
circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 1 ratio_missing:0.0 ratio_per_class: None
circ

In [ ]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):




                        if check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                             missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class=ratio_missing_per_class):


                            print("Experiment already done.")
                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1):
                            print("Not doing this.")
                            continue
                            
                            
                            
                        print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                        missing_data_handling,
                                                                                                                                                          imputation_method, 
                                                                                                                                                          imbalance_ratio,
                                                                                                                                                          missingness_pattern,
                                                                                                                                                          ratio_of_missing_values, 
                                                                                                                                                              ratio_missing_per_class))

                        try:
                            start_time = time()

                            dataset = DatasetGenerator(dataset_name=dataset_name, 
                                                    num_samples=num_samples, 
                                                    imbalance_ratio=imbalance_ratio, 
                                                    missing_data_handling=missing_data_handling,
                                                    imputation_method=imputation_method,
                                                    verbosity=verbosity)

                            # Creation of the missingness
                            dataset.generate_missing_coordinates(missingness_pattern=missingness_pattern, 
                                                                 ratio_of_missing_values=ratio_of_missing_values, 
                                                                 ratio_missing_per_class=ratio_missing_per_class)

                            dataset.split_test_train()

                            # Create the experiments
                            exp = Experiments(dataset_name, 
                                              dataset=dataset, 
                                              purpose='classification', 
                                              approach = approach, 
                                              use_missing_indicator_variables=use_missing_indicator_variables,
                                              previous_experiment=None, 
                                              save_experiment=save, 
                                              verbosity=verbosity, 
                                              debug=False, 
                                              proportion_train=PROPORTION_TRAIN, 
                                              resolution=RESOLUTION, 
                                              bandwidth=BANDWIDTH,
                                              random_state=RANDOM_STATE)


                            # Estimate the distributions 
                            exp.fit()

                            # Estimate the distributions 
                            exp.predict()

                            # Plot results
                            #exp.plot()


                            end_time = time()

                            hours, rest = divmod(end_time - start_time, 3600)
                            minutes, seconds = divmod(rest, 60)
                            print("Done ({}h {}m and {:.2f}s).\n".format(int(hours), int(minutes), seconds))
                        except:
                            print("/!\. Error!\n")
                            print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}\n\n\n".format(dataset_name, 
                                                                                                                                                            missing_data_handling,
                                                                                                                                                              imputation_method, 
                                                                                                                                                              imbalance_ratio,
                                                                                                                                                              missingness_pattern,
                                                                                                                                                              ratio_of_missing_values, 
                                                                                                                                                              ratio_missing_per_class))


Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
Experiment already done.
circles - imputation method: without missing_data_handling: without imbalance: 0.5 pattern: 4 ratio_missing:None ratio_per_class: [0, 0.25]
Doing experiment 2!
